# NN from Assignment 1

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from sklearn.neural_network import MLPClassifier

from sklearn.preprocessing import RobustScaler

from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split

%matplotlib inline

plt.style.use('seaborn')

import warnings
warnings.filterwarnings('ignore')

path = "training_data.csv"

data = pd.read_csv(path)
data = data.sample(frac=1)

target = 'Facies'
features = [feature for feature in list(data.head(0)) if feature not in ("Well Name", "Formation", "Facies")]

X = data[features]

scaler = RobustScaler()
scaler.fit(X)
X = scaler.transform(X)


y = data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

train_sizes = list(range(295,2065,295))

assign1_nn = MLPClassifier(activation='relu', alpha=0.01, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=80, learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

# Training and testing on original data

In [13]:
assign1_nn.fit(X_train,y_train)

y_pred = assign1_nn.predict(X_test)
print(classification_report(y_test,y_pred))

print(sklearn.metrics.accuracy_score(y_test, y_pred))
print(sklearn.metrics.f1_score(y_test, y_pred, average='macro'))

             precision    recall  f1-score   support

          1       0.80      0.59      0.68        63
          2       0.66      0.82      0.73       142
          3       0.81      0.69      0.74       133
          4       0.69      0.74      0.71        42
          5       0.60      0.64      0.62        33
          6       0.67      0.54      0.60        90
          7       0.68      0.83      0.75        18
          8       0.63      0.71      0.67        97
          9       0.92      0.79      0.85        29

avg / total       0.71      0.70      0.70       647

0.7001545595054096
0.7056830367182881


# Training and testing after dimensionality reduction

## PCA

In [22]:
from sklearn.decomposition import PCA

pca4 = PCA(n_components=4)
pca4_transformed = pca4.fit_transform(X_train)
pca4_transformed_test = pca4.transform(X_test)

assign1_nn.fit(pca4_transformed,y_train)

y_pred = assign1_nn.predict(pca4_transformed_test)

print(classification_report(y_test,y_pred))

print(sklearn.metrics.accuracy_score(y_test, y_pred))
print(sklearn.metrics.f1_score(y_test, y_pred, average='macro'))

             precision    recall  f1-score   support

          1       0.70      0.51      0.59        63
          2       0.48      0.71      0.57       142
          3       0.67      0.55      0.60       133
          4       0.30      0.24      0.27        42
          5       0.45      0.30      0.36        33
          6       0.54      0.50      0.52        90
          7       0.62      0.56      0.59        18
          8       0.63      0.64      0.63        97
          9       0.96      0.90      0.93        29

avg / total       0.58      0.57      0.57       647

0.5703245749613601
0.5621893224770671


## ICA

## Randomized Projections

## LDA

# Training and testing after dimensionality reduction and clustering

## PCA and k Means and EM

## ICA and k Means and EM

## Randomized Projections and k Means and EM

## LDA and k Means and EM